1) Define the groud-truth concepts  
2) Train the TabCBM in unsup-concept setting
3) Evaluate performance using several metrics 
4) Supervised-concepts setting

In [52]:
import scipy
import joblib
import sklearn
import numpy as np
import pandas as pd
import os.path as osp
import matplotlib.pyplot as plt

import tensorflow as tf

from tabcbm.models.architectures import construct_encoder, construct_decoder
from tabcbm.models.architectures import construct_end_to_end_model
from tabcbm.models.tabcbm import TabCBM

# Read the data 

In [2]:
# Reading already preprocessed train and test data 

data_dir = 'D:\\PycharmProjects\\AMMISproject\\data\\processed_data'
dataset = 'dataco'

x_train_std = joblib.load(osp.join(data_dir, dataset, 'x_train_std.joblib'))
x_test_std = joblib.load(osp.join(data_dir, dataset, 'x_test_std.joblib'))

x_train = joblib.load(osp.join(data_dir, dataset, 'x_train.joblib'))
x_test = joblib.load(osp.join(data_dir, dataset, 'x_test.joblib'))

y_train = joblib.load(osp.join(data_dir, dataset, 'y_train.joblib'))
y_test = joblib.load(osp.join(data_dir, dataset, 'y_test.joblib'))

print('Shape of the training set: ', x_train_std.shape)
print('Shape of the test set: ', x_test_std.shape)
print('Shape of the trainigb targets: ', y_train.shape)


Shape of the training set:  (138212, 38)
Shape of the test set:  (34553, 38)
Shape of the trainigb targets:  (138212,)


In [54]:
# Defining the concepts 

aggregated_concepts = {
    'Shipment': ['Type', 'Days for shipment (scheduled)', 'Shipping Mode'],
    'Customer': ['Customer Zipcode', 'Customer Segment'],
    'Department': ['Department Name', 'Market'],
    'Store': ['Latitude', 'Longitude'],
    'Order': ['Order Id', 'Order City', 'Order Country', 'order date (DateOrders)',
              'Order Profit Per Order', 'Order Status', 'Sales', 'Order Item Discount',
              'order_year', 'order_month', 'order_day'],
    'ProductCategory': ['Category Name']
}

# In the preprocessed data the naming of the columns differs, so we have to define
# expanded features and put them as a value of a corresponding concept
extended_concepts = {}
for concept, features in aggregated_concepts.items():
    extended_features = []
    for value in features:
            [extended_features.append(column) for column in x_test.columns if value in column]
   
    extended_concepts[concept] = extended_features


In [59]:
# Creating masks for each concept 

concepts_num = len(aggregated_concepts)
total_features_num = x_test.shape[1] 

concepts_masks = pd.DataFrame(0, columns=x_test.columns, index=list(aggregated_concepts.keys()))

for concept, features in extended_concepts.items():
    for feature in features:
        concepts_masks.loc[concept, feature] = 1

In [62]:
# Checking whether the features correspond to the defined concept "Shipment" 

concepts_masks.loc['Shipment', :]

Days for shipment (scheduled)    1
Category Name                    0
Customer Zipcode                 0
Department Name                  0
Latitude                         0
Longitude                        0
Order City                       0
Order Country                    0
Sales                            0
Order Id                         0
Order Item Discount              0
Order Profit Per Order           0
order_year                       0
order_month                      0
order_day                        0
Type_CASH                        1
Type_DEBIT                       1
Type_PAYMENT                     1
Type_TRANSFER                    1
Customer Segment_Consumer        0
Customer Segment_Corporate       0
Customer Segment_Home Office     0
Market_Africa                    0
Market_Europe                    0
Market_LATAM                     0
Market_Pacific Asia              0
Market_USCA                      0
Shipping Mode_First Class        1
Shipping Mode_Same D

## Create main components for TabCBM

In [63]:
# Parameters defining the architecture we will use

input_shape = x_train_std.shape[1:]
num_outputs = len(set(y_train))
encoder_units = [16, 16]
decoder_units = [16]
latent_dims = 16
learning_rate = 0.001
validation_size = 0.1

print('Input shape: ', input_shape)
print('Number of outputs: ', num_outputs)

Input shape:  (38,)
Number of outputs:  2


In [64]:
# Next, we build the feature to latent code encoder model (i.e., phi)
encoder = construct_encoder(input_shape, encoder_units, latent_dims)

In [65]:
# Then, we build the concept to label model  (i.e., the label predictor f)

decoder_inputs = tf.keras.Input(shape=[latent_dims])
decoder_graph = construct_decoder(decoder_units, num_outputs)
decoder = tf.keras.Model(
    decoder_inputs,
    decoder_graph(decoder_inputs),
    name="decoder",
)

In [66]:
# We then put them both together to make an end-to-end model we can pretrain

end_to_end_model, encoder, decoder = construct_end_to_end_model(input_shape,
                                                                encoder,
                                                                decoder,
                                                                num_outputs,
                                                                learning_rate)

end_to_end_model.summary()

pretrain_epochs = 30
batch_size = 512
pretrain_hist = end_to_end_model.fit(
    x=x_train_std,
    y=y_train,
    epochs=pretrain_epochs,
    batch_size=batch_size,
    validation_split=validation_size,
    verbose=1,
)

Model: "complete_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)     │ (None, 38)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 16)             │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 1)              │           289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,457 (5.69 KB)

 Trainable params: 1,457 (5.69 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
243/243 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - binary_accuracy: 0.5762 - loss: 0.6290 - val_binary_accuracy: 0.6929 - val_loss: 0.5467
Epoch 2/30
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6935 - loss: 0.5431 - val_binary_accuracy: 0.6945 - val_loss: 0.5430
Epoch 3/30
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6980 - loss: 0.5372 - val_binary_accuracy: 0.6935 - val_loss: 0.5421
Epoch 4/30
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6982 - loss: 0.5378 - val_binary_accuracy: 0.6949 - val_loss: 0.5419
Epoch 5/30
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6963 - loss: 0.5379 - val_binary_accuracy: 0.6952 - val_loss: 0.5412
Epoch 6/30
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6980 - loss: 0.5358 - val_binary_accuracy: 0.6948 - val_loss: 0.5409
Epoch 7/30
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6983 - loss: 0.5358 - val_binary_accuracy: 0.6950 - val_loss: 0.5410
Epoch 

In [67]:
# Evaluate pretrained model

# We will accumulate all metrics/results in the same dictionary
results = {}

end_to_end_preds = end_to_end_model.predict(
    x_test_std,
    batch_size=batch_size,
)

# We assume that we have outputed logits
if np.min(end_to_end_preds) < 0.0 or np.max(end_to_end_preds) > 1:
    end_to_end_preds = tf.math.sigmoid(end_to_end_preds).numpy()
end_to_end_preds = (end_to_end_preds >= 0.5).astype(np.int32)
results['pre_train_acc'] = sklearn.metrics.accuracy_score(
    y_test,
    end_to_end_preds,
)
results['pre_train_auc'] = sklearn.metrics.roc_auc_score(
    y_test,
    end_to_end_preds,
)
print(f"Pretrained model task accuracy: {results['pre_train_acc']*100:.2f}%")


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Pretrained model task accuracy: 69.39%


# Construct TabCBM

For this, we will first compute the empirical covariance matrix in order for us to learn useful masks using a similar approach to that proposed by SEFS:

In [68]:
# Construct the training set's empirical covariance matrix
# NOTE: This step can be very computationally expensive/intractable in large
#       datasets. In those cases, one may ignore the covariance matrix when
#       performing TabCBM's pretraining at the potential cost of performance or
#       more accurate concept discovery.
cov_mat = np.corrcoef(x_train_std.T)
print(cov_mat)

[[ 1.00000000e+00  1.00105563e-03  8.38634556e-03 ... -3.34490471e-03
   6.45576389e-03  7.41082360e-04]
 [ 1.00105563e-03  1.00000000e+00  2.93211877e-03 ...  1.57344435e-03
  -2.49050682e-03 -4.45254638e-03]
 [ 8.38634556e-03  2.93211877e-03  1.00000000e+00 ...  3.03096663e-03
   5.79808624e-03  1.24462552e-03]
 ...
 [-3.34490471e-03  1.57344435e-03  3.03096663e-03 ...  1.00000000e+00
  -1.99639348e-01 -1.38666258e-01]
 [ 6.45576389e-03 -2.49050682e-03  5.79808624e-03 ... -1.99639348e-01
   1.00000000e+00 -2.07627716e-01]
 [ 7.41082360e-04 -4.45254638e-03  1.24462552e-03 ... -1.38666258e-01
  -2.07627716e-01  1.00000000e+00]]


In [69]:
# Number of concepts we want to discover
n_concepts = 6

# Set the weights for the different regularisers in the loss
coherence_reg_weight = 0.1  # $lambda_{co}
diversity_reg_weight = 5  # $lambda_{div}
feature_selection_reg_weight = 5  # $lambda_{spec}
gate_estimator_weight = 10  # Gate prediction regularizer for SEFS's pre-text task

# Select how many neighbors to use for the coherency loss (must be less than the batch size!)
top_k = 256

In [70]:
# Generate a dictionary with the parameters to use for TabCBM as we will have
# to use the same parameters twice:
tab_cbm_params = dict(
    features_to_concepts_model=encoder,  # The $\phi$ sub-model
    concepts_to_labels_model=decoder,  # The $f$ sub-model
    latent_dims=latent_dims,  # The dimensionality of the concept embeddings $m$
    n_concepts=n_concepts,  # The number of concepts to discover $k^\prime$
    cov_mat=cov_mat,  # The empirical covariance matrix
    loss_fn=end_to_end_model.loss,  # The downstream task loss function
    # Then we provide all the regularizers weights
    coherence_reg_weight=coherence_reg_weight,
    diversity_reg_weight=diversity_reg_weight,
    feature_selection_reg_weight=feature_selection_reg_weight,
    gate_estimator_weight=gate_estimator_weight,
    top_k=top_k,

    # And indicate that we will not be providing any supervised concepts! Change
    # this is training concepts (e.g., `c_train`) are provided/known during
    # training
    n_supervised_concepts=0,
    concept_prediction_weight=0,

    # The accuracy metric to use for logging performance
    acc_metric=(
        lambda y_true, y_pred: tf.math.reduce_mean(
            tf.keras.metrics.sparse_categorical_accuracy(
                y_true,
                y_pred,
            )
        )
    ),

    # ANd architectural details of the self-supervised reconstruction modules
    concept_generator_units=[64],
    rec_model_units=[64],
)

In [13]:
# Mask Generator Self-supervised Training

# Next, we proceed to do the SELF-SUPERVISED TRAINING of the MASK
# GENERATORS for TabCBM. For this, we will follow a similar approach
# to that of SEFS. Our TabCBM module allows one to do this by setting
# the self_supervised_mode flag to True before calling the .fit() method:

# We can now construct our TabCBM model which we will first self-supervise!
ss_tabcbm = TabCBM(self_supervised_mode=True,  **tab_cbm_params)
ss_tabcbm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate,))
ss_tabcbm.summary()


[WARNING] Assuming independence between features in TabCBM training.


d:\PycharmProjects\AMMISproject\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "tab_cbm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ decoder (Functional)            │ (None, 1)              │           289 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 16)             │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 16)             │        11,516 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concept_generators_0            │ ?                      │   0 (unbuilt) │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rec_values_model_0 (Sequential) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rec_mask_model (Sequential)     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concept_generators_1            │ ?                      │   0 (unbuilt) │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concept_generators_2            │ ?                      │   0 (unbuilt) │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concept_generators_3            │ ?                      │   0 (unbuilt) │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concept_generators_4            │ ?                      │   0 (unbuilt) │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concept_generators_5            │ ?                      │   0 (unbuilt) │
│ (Sequential)                    │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,429 (52.46 KB)

 Trainable params: 13,201 (51.57 KB)

 Non-trainable params: 228 (912.00 B)

In [71]:
self_supervised_train_epochs = 50
print("TabCBM self-supervised training stage...")
ss_tabcbm_hist = ss_tabcbm.fit(
    x=x_train_std,
    y=y_train,
    validation_split=validation_size,
    epochs=self_supervised_train_epochs,
    batch_size=batch_size,
    verbose=1,
)

print("\tTabCBM self-supervised training completed")

TabCBM self-supervised training stage...
Epoch 1/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - avg_concept_size: 0.0000e+00 - avg_features_rec_loss: 0.7823 - avg_mask_rec_loss: 26.4700 - loss: 163.5136 - max_probability: 0.0000e+00 - mean_probability: 0.0000e+00 - min_probability: 0.0000e+00 - prob_sparsity_loss: 0.0000e+00 - reg_loss_closest: 0.0000e+00 - reg_loss_similarity: 0.0000e+00 - task_loss: 0.0000e+00 - val_avg_features_rec_loss: 0.8006 - val_avg_mask_rec_loss: 6.7000 - val_loss: 45.0035
Epoch 2/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - avg_concept_size: 0.0000e+00 - avg_features_rec_loss: 0.8057 - avg_mask_rec_loss: 6.6508 - loss: 44.7388 - max_probability: 0.0000e+00 - mean_probability: 0.0000e+00 - min_probability: 0.0000e+00 - prob_sparsity_loss: 0.0000e+00 - reg_loss_closest: 0.0000e+00 - reg_loss_similarity: 0.0000e+00 - task_loss: 0.0000e+00 - val_avg_features_rec_loss: 0.8222 - val_avg_mask_rec_loss: 6.5372 - val_loss:

In [16]:
# First we will instantiate a new TabCBM that is NOT in self-supervised mode,
# and we will load its weights so that they are the same as the model whose
# mask generators have been pre-trained using the SS loss.
tabcbm_supervised = TabCBM(
    self_supervised_mode=False,
    # Notice how we provide as concept generators the concept generators of the
    # SS TabCBM:
    concept_generators=ss_tabcbm.concept_generators,
    # as well as the feature probability masks:
    prior_masks=ss_tabcbm.feature_probabilities,
    **tab_cbm_params,
)
tabcbm_supervised.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
tabcbm_supervised.summary()

[WARNING] Assuming independence between features in TabCBM training.


d:\PycharmProjects\AMMISproject\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "tab_cbm_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ decoder (Functional)            │ (None, 1)              │           289 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 16)             │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (None, 16)             │        11,516 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concept_generators_0            │ (None, 16)             │         3,536 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concept_generators_1            │ (None, 16)             │         3,536 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concept_generators_2            │ (None, 16)             │         3,536 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concept_generators_3            │ (None, 16)             │         3,536 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concept_generators_4            │ (None, 16)             │         3,536 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concept_generators_5            │ (None, 16)             │         3,536 │
│ (Sequential)                    │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,645 (135.33 KB)

 Trainable params: 34,417 (134.44 KB)

 Non-trainable params: 228 (912.00 B)

In [17]:
max_epochs = 50   
print("TabCBM self-supervised training stage...")

tabcbm_hist = tabcbm_supervised.fit(
    x=x_train_std,
    y=y_train,
    validation_split=validation_size,
    epochs=max_epochs,
    batch_size=batch_size,
    verbose=1,
)

print("\tTabCBM supervised training completed")

TabCBM self-supervised training stage...
Epoch 1/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.4257 - avg_concept_size: 17.5381 - avg_features_rec_loss: 0.0000e+00 - avg_mask_rec_loss: 0.0000e+00 - loss: 3.2145 - max_probability: 0.7157 - mean_probability: 0.4860 - min_probability: 0.2578 - prob_sparsity_loss: 2.4302 - reg_loss_closest: 0.0125 - reg_loss_similarity: 0.2339 - task_loss: 0.5629 - val_accuracy: 0.4157 - val_loss: 3.0219 - val_prob_sparsity_loss: 2.2212 - val_reg_loss_closest: 0.0151 - val_reg_loss_similarity: 0.2326 - val_task_loss: 0.5832
Epoch 2/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.4272 - avg_concept_size: 13.5139 - avg_features_rec_loss: 0.0000e+00 - avg_mask_rec_loss: 0.0000e+00 - loss: 2.9097 - max_probability: 0.6621 - mean_probability: 0.4306 - min_probability: 0.2157 - prob_sparsity_loss: 2.1528 - reg_loss_closest: 0.0156 - reg_loss_similarity: 0.2327 - task_loss: 0.5398 - val_accuracy: 0.4157 - val_loss: 2.7505 - val_prob_sparsi

# Evaluate TabCBM

In [18]:
test_y_pred, test_concept_scores = tabcbm_supervised.predict(
    x_test_std,
    batch_size=batch_size,
)

68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [34]:
(tf.math.sigmoid(test_y_pred).numpy() > 0.5).sum(), test_y_pred.shape[0]

(12050, 34553)

In [38]:
if np.min(test_y_pred) < 0.0 or np.max(test_y_pred) > 1:
        # Then we assume that we have outputed logits
        test_preds = tf.math.sigmoid(test_y_pred).numpy()

test_preds = (test_preds > 0.5).astype(np.int32)
results['acc'] = sklearn.metrics.accuracy_score(
    y_test,
    test_preds,
)
results['auc'] = sklearn.metrics.roc_auc_score(
    y_test,
    test_preds,
)

print(f"Accuracy is {results['acc']*100:.2f}%")

Accuracy is 69.67%


In [39]:
test_concept_scores

array([[0.49567437, 0.56690246, 0.5377383 , 0.4823982 , 0.71265817,
        0.5291953 ],
       [0.5012155 , 0.49902573, 0.4999965 , 0.50061405, 0.5238639 ,
        0.56576735],
       [0.5710136 , 0.6701345 , 0.5800061 , 0.5610418 , 0.4773939 ,
        0.44498098],
       ...,
       [0.50121325, 0.5031216 , 0.5003134 , 0.5001966 , 0.5243295 ,
        0.56199974],
       [0.5009029 , 0.5030297 , 0.50002974, 0.49985534, 0.5234854 ,
        0.5568946 ],
       [0.5006396 , 0.4971397 , 0.49987555, 0.5003974 , 0.53039616,
        0.55005425]], dtype=float32)

In [28]:
tabcbm_supervised.feature_probabilities

<KerasVariable shape=(6, 38), dtype=float32, path=tab_cbm_2/probability_vector_logits>

## Comparison of the calculated and ground-truth masks 

In [26]:
# The masks are stored as logits, so we need to turn them to probabilities using
# a sigmoid
masks = tf.sigmoid(tabcbm_supervised.feature_probabilities).numpy()
print("Thresholded concept masks learnt by TabCBM:")
for i, mask in enumerate((masks>0.1).astype(np.int32)):
    print("\tFor concept", i, "we are selecting the following features", mask)
print("-" * 80)
print("-" * 80)

Thresholded concept masks learnt by TabCBM:
	For concept 0 we are selecting the following features [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
	For concept 1 we are selecting the following features [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
	For concept 2 we are selecting the following features [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
	For concept 3 we are selecting the following features [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
	For concept 4 we are selecting the following features [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
	For concept 5 we are selecting the following features [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
--------------------------------------------------------------------------------
-----------------------------------------------------------------------

# Load pretrained TabCBM 

In [40]:
tabcbm_pretrained = tf.keras.models.load_model(r'D:\\PycharmProjects\\AMMISproject\\trained_tabcbm_4concepts_1500epochs\\tabcbm_supervised.keras')


TypeError: <class 'tabcbm.models.tabcbm.TabCBM'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'tabcbm.models.tabcbm', 'class_name': 'TabCBM', 'config': {'name': 'tab_cbm_1', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}, 'registered_name': 'TabCBM', 'compile_config': {'loss': None, 'loss_weights': None, 'metrics': None, 'weighted_metrics': None, 'run_eagerly': False, 'steps_per_execution': 1, 'jit_compile': False}}.

Exception encountered: Unable to revive model from config. When overriding the `get_config()` method, make sure that the returned config contains all items used as arguments in the  constructor to <class 'tabcbm.models.tabcbm.TabCBM'>, which is the default behavior. You can override this default beha